## Import

In [63]:
import datetime
import json
import math
import os
import numpy as np
import requests
from numpy import mean, array
from numpy import std
import pandas as pd
from datetime import date
# import arrow
import pwlf
import math
from datetime import datetime

import seaborn as sns
from numpy.polynomial.polynomial import polyfit
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import figure
import matplotlib as mlp
from scipy import stats
import scipy

from math import log
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle
# import chardet

from mlxtend.feature_selection import SequentialFeatureSelector as sfs, SequentialFeatureSelector
from matplotlib import pyplot as plt, pyplot
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold, StratifiedKFold, KFold, cross_val_predict
from sklearn.model_selection import train_test_split, cross_val_score
# from xgboost import XGBRegressor
# import xgboost as xgb
# import shap

In [64]:
path_code = os.path.dirname(os.path.realpath("file"))
path_files = path_code.replace('\\code\\Data processing', '\\data')

### Read Files (some are processed?)

In [ ]:

counting_2022_2023 = pd.read_excel(path_files + "\\Raw data\\counting_2022_2023_MB-20240818.xlsx", sheet_name="לתמריקה (3)")
counting_2022_2023.dropna(axis = 0, how = 'all', inplace = True)
counting_2022_2023.rename(columns={ 'HAVA:PLOT': 'file_name'}, inplace=True)


processed_data_2012_2021 = pd.read_excel(path_files + "\\Processed data\\processed_data_2012_2020_final_hopefully.xlsx")
processed_counting_2022_2023 = pd.read_excel(path_files + "\\Processed data\\counting_2022_2023_processed.xlsx")
processed_data = pd.concat([processed_data_2012_2021, processed_counting_2022_2023])
# processed_data = pd.read_excel(path_files + "\\Processed data\\processed_data_2012_2023.xlsx")
# homogemic_df = pd.read_excel(path_files + "\\Processed data\\Homogenic plots 2012-2023.xlsx")
# heterogenic_df = pd.read_excel(path_files + "\\Processed data\\Heterogenic plots 2012-2023.xlsx")


## MAIN

In [ ]:

processed_data_quality = quality_processing(processed_data)
processed_data_quality.to_excel(path_files + "\\Processed data\\YS_processed_data_2012_2023.xlsx")
# processed_data_quality = quality_processing(heterogenic_df)
# processed_data_quality.to_excel(path_files + "\\Processed data\\Heterogenic plots 2012-2023.xlsx")
# trees_age = pd.read_excel(path_files + "\\Processed data\\Trees per plot.xlsx")
# trees_age = trees_age[["HAVA:PLOT", "שנת נטיעה"]]
# processed_data = pd.merge(processed_data,trees_age, left_on="Farm:Plot", right_on="HAVA:PLOT")
# processed_data = processed_data.drop('HAVA:PLOT', axis=1)
# processed_data = quality_processing(processed_data)
# processed_data.rename(columns={'שנת נטיעה': 'Planting year'}, inplace=True)
# processed_data['Age'] = processed_data['Year'] - processed_data['Planting year']
# processed_data.to_excel(path_files + "\\Processed data\\processed_data_2012_2023.xlsx")
# df_results = basic_model(processed_data, feature_dict, quality_labels)
# df_results.to_excel(path_files + "\\Model results\\All plots (without age as feature)\\Quality model results 2012-2023 without age_2.xlsx")

# df_results = basic_model(homogemic_df, feature_dict_with_age, quality_labels)
# df_results.to_excel(path_files + "\\Model results\\Homogenic plots (with age as feature)\\Quality model results 2012-2023 with age.xlsx")
# all_features_without_monthly_with_age = all_features_without_monthly.copy()
# all_features_without_monthly_with_age.append("Age")
# counts_with_age = counts.copy()
# counts_with_age.append("Age")
# Feature_analysis(processed_data, counts, "Reslative_yield_per_tree")

## Processing

### Function: initial_processing_counting

In [15]:
def initial_processing_counting(original_df):
    processed_df = pd.DataFrame()

    for farm_plot in original_df['file_name'].unique():
        for date in original_df.loc[original_df['file_name'] == farm_plot]['DAY'].unique():

            temp_df = original_df.loc[(original_df['file_name'] == farm_plot) & (original_df['DAY'] == date)]

            # Get days since January
            year = date[0:4]
            a = arrow.get(f'{year}-01-01') #january 1st of that year
            b = arrow.get(date[0:10]) # the date
            delta = (b - a)

            AVG_fruitlets_upper = float("NaN")
            AVG_fruitlets_center = float("NaN")
            AVG_fruitlets_lower = float("NaN")

            num_samples_upper = 0
            num_samples_center = 0
            num_samples_lower = 0

            # UPPER
            # If bunch A and bunch B both aren't empty (if not all values are nan in both of them):
            if not (temp_df['BUNCHA'].isnull().values.all()) | (temp_df['BUNCHB'].isnull().values.all()):
                num_samples_upper = 2
                AVG_fruitlets_upper = ((temp_df['QA1'].mean() + temp_df['QA2'].mean() + temp_df['QA3'].mean() + temp_df['QA4'].mean() +
                                        temp_df['QB1'].mean() + temp_df['QB2'].mean() + temp_df['QB3'].mean() +
                                        temp_df['QB4'].mean()) / 8) * ((temp_df['BUNCHA'].mean() + temp_df['BUNCHB'].mean()) / 2)
            # If bunch A isn't empty but bunch B is:
            elif not (temp_df['BUNCHA'].isnull().values.all()):
                num_samples_upper = 1
                AVG_fruitlets_upper = ((temp_df['QA1'].mean() + temp_df['QA2'].mean() + temp_df['QA3'].mean() + temp_df['QA4'].mean())
                                       / 4) * temp_df['BUNCHA'].mean()

            # If bunch B isn't empty but bunch A is:
            elif not (temp_df['BUNCHB'].isnull().values.all()):
                num_samples_upper = 1
                AVG_fruitlets_upper = ((temp_df['QB1'].mean() + temp_df['QB2'].mean() + temp_df['QB3'].mean() + temp_df['QB4'].mean())
                                       / 4) * temp_df['BUNCHB'].mean()


            # CENTER

            if not (temp_df['BUNCHC'].isnull().values.all()) | (temp_df['BUNCHD'].isnull().values.all()):
                num_samples_center = 2
                AVG_fruitlets_center = ((temp_df['QC1'].mean() + temp_df['QC2'].mean() + temp_df['QC3'].mean() + temp_df['QC4'].mean() +
                                        temp_df['QD1'].mean() + temp_df['QD2'].mean() + temp_df['QD3'].mean() +
                                        temp_df['QD4'].mean()) / 8) * ( (temp_df['BUNCHC'].mean() + temp_df['BUNCHD'].mean()) / 2)

            elif not (temp_df['BUNCHC'].isnull().values.all()):
                num_samples_center = 1
                AVG_fruitlets_center = ((temp_df['QC1'].mean() + temp_df['QC2'].mean() + temp_df['QC3'].mean() +
                                        temp_df['QC4'].mean()) / 4) * temp_df['BUNCHC'].mean()

            elif not (temp_df['BUNCHD'].isnull().values.all()):
                num_samples_center = 1
                AVG_fruitlets_center = ((temp_df['QD1'].mean() + temp_df['QD2'].mean() + temp_df['QD3'].mean() +
                                        temp_df['QD4'].mean()) / 4) * temp_df['BUNCHD'].mean()

            # LOWER

            if not (temp_df['BUNCHE'].isnull().values.all()) | (temp_df['BUNCHF'].isnull().values.all()):
                num_samples_lower = 2
                AVG_fruitlets_lower = ((temp_df['QE1'].mean() + temp_df['QE2'].mean() + temp_df['QE3'].mean() + temp_df['QE4'].mean() +
                                        temp_df['QF1'].mean() + temp_df['QF2'].mean() + temp_df['QF3'].mean() +
                                        temp_df['QF4'].mean()) / 8) * ( (temp_df['BUNCHE'].mean() + temp_df['BUNCHF'].mean()) / 2)
            elif not (temp_df['BUNCHE'].isnull().values.all()):
                num_samples_lower = 1
                AVG_fruitlets_lower = ((temp_df['QE1'].mean() + temp_df['QE2'].mean() + temp_df['QE3'].mean() +
                                        temp_df['QE4'].mean()) / 4) * temp_df['BUNCHE'].mean()
            elif not (temp_df['BUNCHF'].isnull().values.all()):
                num_samples_lower = 1
                AVG_fruitlets_lower = ((temp_df['QF1'].mean() + temp_df['QF2'].mean() + temp_df['QF3'].mean() +
                                        temp_df['QF4'].mean()) / 4) * temp_df['BUNCHF'].mean()

            # If we have all three (upper, center, lower):

            if not (math.isnan(AVG_fruitlets_upper)) | (math.isnan(AVG_fruitlets_upper)) | (math.isnan(AVG_fruitlets_upper)):
                SUM_tree_fruitlets =  temp_df['TBRANCH'].mean() * (1/4 * AVG_fruitlets_upper + 1/2 * AVG_fruitlets_center + 1/4 * AVG_fruitlets_lower)

            else:
                SUM_tree_fruitlets = float("NaN")


            processed_df = processed_df.append({'Farm:Plot': farm_plot, 'Year' :year, 'Date':date[0:10], 'Days since january 1st': delta.days,
                                                'AVG_clusters': temp_df['TBRANCH'].mean(), 'AVG_fruitlets_upper': AVG_fruitlets_upper,
                                                'NUM_samples_upper' : num_samples_upper, 'AVG_fruitlets_center': AVG_fruitlets_center,
                                                'NUM_samples_center' : num_samples_center,'AVG_fruitlets_lower': AVG_fruitlets_lower,
                                                'NUM_samples_lower' : num_samples_lower,'SUM_tree_fruitlets':SUM_tree_fruitlets},
                                               ignore_index=True)
    return processed_df

def initial_processing_counting_6_samples(original_df):
    processed_df = pd.DataFrame()

    original_df['DAY'] = original_df['DAY'].astype(str)

    for farm_plot in original_df['file_name'].unique():
        for date in original_df.loc[original_df['file_name'] == farm_plot]['DAY'].unique():

            temp_df = original_df.loc[(original_df['file_name'] == farm_plot) & (original_df['DAY'] == date)]

            # Get days since January
            year = date[0:4]
            month = date[4:6]
            day = date[6:8]
            a = arrow.get(f'{year}-01-01') #january 1st of that year
            b = arrow.get(f'{year}-{month}-{day}') # the date
            delta = (b - a)
            date_string = f'{year}-{month}-{day}'

            AVG_fruitlets_upper = float("NaN")
            AVG_fruitlets_center = float("NaN")
            AVG_fruitlets_lower = float("NaN")

            num_samples_upper = 0
            num_samples_center = 0
            num_samples_lower = 0

            # UPPER
            # If bunch A and bunch B both aren't empty (if not all values are nan in both of them):
            if not (temp_df['BUNCHA'].isnull().values.all()) | (temp_df['BUNCHB'].isnull().values.all()):
                num_samples_upper = 2
                AVG_fruitlets_upper = ((temp_df['QA1'].mean() + temp_df['QA2'].mean() + temp_df['QA3'].mean() + temp_df['QA4'].mean() +
                                        temp_df['QA5'].mean() + temp_df['QA6'].mean() +
                                        temp_df['QB1'].mean() + temp_df['QB2'].mean() + temp_df['QB3'].mean() +
                                        temp_df['QB4'].mean()+ temp_df['QB5'].mean()+
                                        temp_df['QB6'].mean() ) / 12) * ((temp_df['BUNCHA'].mean() + temp_df['BUNCHB'].mean()) / 2)
            # If bunch A isn't empty but bunch B is:
            elif not (temp_df['BUNCHA'].isnull().values.all()):
                num_samples_upper = 1
                AVG_fruitlets_upper = ((temp_df['QA1'].mean() + temp_df['QA2'].mean() + temp_df['QA3'].mean() + temp_df['QA4'].mean()
                                        + temp_df['QA5'].mean() + temp_df['QA6'].mean())
                                       / 6) * temp_df['BUNCHA'].mean()

            # If bunch B isn't empty but bunch A is:
            elif not (temp_df['BUNCHB'].isnull().values.all()):
                num_samples_upper = 1
                AVG_fruitlets_upper = ((temp_df['QB1'].mean() + temp_df['QB2'].mean() + temp_df['QB3'].mean() + temp_df['QB4'].mean()
                                        + temp_df['QB5'].mean() + temp_df['QB6'].mean())
                                       / 6) * temp_df['BUNCHB'].mean()


            # CENTER

            if not (temp_df['BUNCHC'].isnull().values.all()) | (temp_df['BUNCHD'].isnull().values.all()):
                num_samples_center = 2
                AVG_fruitlets_center = ((temp_df['QC1'].mean() + temp_df['QC2'].mean() + temp_df['QC3'].mean() + temp_df['QC4'].mean() +
                                         temp_df['QC5'].mean() + temp_df['QC6'].mean() +
                                        temp_df['QD1'].mean() + temp_df['QD2'].mean() + temp_df['QD3'].mean() + temp_df['QD4'].mean() +
                                         temp_df['QD5'].mean() +
                                        temp_df['QD6'].mean() ) / 12) * ( (temp_df['BUNCHC'].mean() + temp_df['BUNCHD'].mean()) / 2)

            elif not (temp_df['BUNCHC'].isnull().values.all()):
                num_samples_center = 1
                AVG_fruitlets_center = ((temp_df['QC1'].mean() + temp_df['QC2'].mean() + temp_df['QC3'].mean() +
                                         temp_df['QC4'].mean() + temp_df['QC5'].mean() +
                                        temp_df['QC6'].mean()) / 6) * temp_df['BUNCHC'].mean()

            elif not (temp_df['BUNCHD'].isnull().values.all()):
                num_samples_center = 1
                AVG_fruitlets_center = ((temp_df['QD1'].mean() + temp_df['QD2'].mean() + temp_df['QD3'].mean() +
                                         temp_df['QD4'].mean() + temp_df['QD5'].mean() +
                                        temp_df['QD6'].mean()) / 6) * temp_df['BUNCHD'].mean()

            # LOWER

            if not (temp_df['BUNCHE'].isnull().values.all()) | (temp_df['BUNCHF'].isnull().values.all()):
                num_samples_lower = 2
                AVG_fruitlets_lower = ((temp_df['QE1'].mean() + temp_df['QE2'].mean() + temp_df['QE3'].mean() + temp_df['QE4'].mean() +
                                        temp_df['QE5'].mean() + temp_df['QE6'].mean() +
                                        temp_df['QF1'].mean() + temp_df['QF2'].mean() + temp_df['QF3'].mean() +
                                        temp_df['QF4'].mean() + temp_df['QF5'].mean() +
                                        temp_df['QF6'].mean()) / 12) * ( (temp_df['BUNCHE'].mean() + temp_df['BUNCHF'].mean()) / 2)
            elif not (temp_df['BUNCHE'].isnull().values.all()):
                num_samples_lower = 1
                AVG_fruitlets_lower = ((temp_df['QE1'].mean() + temp_df['QE2'].mean() + temp_df['QE3'].mean() +
                                        temp_df['QE4'].mean() + temp_df['QE5'].mean() +
                                        temp_df['QE6'].mean()) / 6) * temp_df['BUNCHE'].mean()
            elif not (temp_df['BUNCHF'].isnull().values.all()):
                num_samples_lower = 1
                AVG_fruitlets_lower = ((temp_df['QF1'].mean() + temp_df['QF2'].mean() + temp_df['QF3'].mean() +
                                        temp_df['QF4'].mean() + temp_df['QF5'].mean() +
                                        temp_df['QF6'].mean()) / 6) * temp_df['BUNCHF'].mean()

            # If we have all three (upper, center, lower):

            if not (math.isnan(AVG_fruitlets_upper)) | (math.isnan(AVG_fruitlets_upper)) | (math.isnan(AVG_fruitlets_upper)):
                SUM_tree_fruitlets =  temp_df['TBRANCH'].mean() * (1/4 * AVG_fruitlets_upper + 1/2 * AVG_fruitlets_center + 1/4 * AVG_fruitlets_lower)

            else:
                SUM_tree_fruitlets = float("NaN")


            processed_df = processed_df.append({'Farm:Plot': farm_plot, 'Year' :year, 'Date':date_string, 'Days since january 1st': delta.days,
                                                'AVG_clusters': temp_df['TBRANCH'].mean(), 'AVG_fruitlets_upper': AVG_fruitlets_upper,
                                                'NUM_samples_upper' : num_samples_upper, 'AVG_fruitlets_center': AVG_fruitlets_center,
                                                'NUM_samples_center' : num_samples_center,'AVG_fruitlets_lower': AVG_fruitlets_lower,
                                                'NUM_samples_lower' : num_samples_lower,'SUM_tree_fruitlets':SUM_tree_fruitlets},
                                               ignore_index=True)
    return processed_df


### Function: weather_data, weather_data_Flowering_periods 

In [66]:
def weather_data(): #processed_df
    print('a')
    # df = pd.read_excel(path_files + "\\Processed data\\Combined Meteorological data.xlsx")
    print('b')

    df_2011 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2011 radiation.xlsx')
    df_2012 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2012 radiation.xlsx')
    df_2013 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2013 radiation.xlsx')
    df_2014 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2014 radiation.xlsx')
    df_2015 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2015 radiation.xlsx')
    df_2016 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2016 radiation.xlsx')
    df_2017 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2017 radiation.xlsx')
    df_2018 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2018 radiation.xlsx')
    df_2019 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2019 radiation.xlsx')
    df_2020 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2020 radiation.xlsx')
    df_2021 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2021 radiation.xlsx')
    df_2022 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2022 radiation.xlsx')
    df_2023 = pd.read_excel(path_files + '\\Raw data/Meteorology Yotvata 2023 radiation.xlsx')
    df_list = [df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023]
    # df_list = [df_2013]
    df = pd.concat(df_list)
    #df.to_excel(path_files + "\\Processed data\\Combined Meteorological data 2012 2023.xlsx")
    df = df.drop_duplicates()
    df.rename(columns={'תחנה': 'Station',
                      'תאריך ושעה (שעון חורף)': 'Date & Time (Winter)',
                      'לחות יחסית (%)': 'Relative humidity (%)',
                      'טמפרטורה (C°)': 'Temperature (°C)',
                      'כמות גשם (מ"מ)': 'Rainfall (mm)',
                      'קרינה ישירה (וואט/מ"ר)': 'Radiation (W/m2)'}, inplace=True)


    df['Date'] = pd.to_datetime(df['Date & Time (Winter)'], dayfirst=True).dt.date
    df['Time'] = pd.to_datetime(df['Date & Time (Winter)'], dayfirst=True).dt.time
    df['hour'] = pd.to_datetime(df['Date & Time (Winter)'], dayfirst=True).dt.hour
    df['month'] = pd.to_datetime(df['Date & Time (Winter)'], dayfirst=True).dt.month
    df['year'] = pd.to_datetime(df['Date & Time (Winter)'], dayfirst=True).dt.year

    df['Temperature (°C)'] = pd.to_numeric(df['Temperature (°C)'], errors='coerce')
    df['Relative humidity (%)'] = pd.to_numeric(df['Relative humidity (%)'], errors='coerce')
    df.to_excel(path_files + "\\Processed data\\Combined Meteorological data 2011 2023.xlsx")

    heat_thresholod = 18

    conditions = [
        (df['Temperature (°C)'] > heat_thresholod),
        (df['Temperature (°C)'] <= heat_thresholod)
    ]

    values = [(df['Temperature (°C)'] - heat_thresholod) * 0.16666667, 0]

    df['degree_hours_10_min'] = np.select(conditions, values)

    df_temperature = df.groupby(['Station', 'year', 'month']).degree_hours_10_min.sum().reset_index()
    df_temperature = df_temperature.pivot(index=['Station', 'year'], columns='month', values='degree_hours_10_min')
    df_temperature.columns = ['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12']
    df_temperature['t10'] = df_temperature['t10'].shift(periods=1)
    df_temperature['t11'] = df_temperature['t11'].shift(periods=1)
    df_temperature['t12'] = df_temperature['t12'].shift(periods=1)

    df_humidity = df.groupby(['Station', 'year', 'month'])['Relative humidity (%)'].mean().reset_index()
    df_humidity = df_humidity.pivot(index=['Station', 'year'], columns='month', values='Relative humidity (%)')
    df_humidity.columns = ['h1',
                           'h2',
                           'h3',
                           'h4',
                           'h5',
                           'h6',
                           'h7',
                           'h8',
                           'h9',
                           'h10',
                           'h11',
                           'h12']

    df_humidity['h10'] = df_humidity['h10'].shift(periods=1)
    df_humidity['h11'] = df_humidity['h11'].shift(periods=1)
    df_humidity['h12'] = df_humidity['h12'].shift(periods=1)

    df_meteorology = pd.merge(df_temperature, df_humidity, on='year', how='outer')

    #final_df = pd.merge(processed_df, df_meteorology, left_on='Year', right_on='year', how='outer')

    return df_meteorology #final_df

def weather_data_Flowering_periods(meteorological_data): #processed_df_weather

    df_periods = pd.DataFrame()

    # processed_df_weather = processed_df_weather[processed_df_weather['Year'].notna()]
    # processed_df_weather['Year'] = processed_df_weather['Year'].astype('int64')

    heat_thresholod = 18

    conditions = [
        (meteorological_data['Temperature (°C)'] > heat_thresholod),
        (meteorological_data['Temperature (°C)'] <= heat_thresholod)
    ]

    values = [(meteorological_data['Temperature (°C)'] - heat_thresholod) * 0.16666667, 0]

    meteorological_data['degree_hours_10_min'] = np.select(conditions, values)

    for i in meteorological_data['year'].unique():
        df_periods = pd.concat([df_periods, pd.DataFrame([{'year': i, "T_Inf_differentiation": meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i-1}-11-01',
                                                                               format='%Y-%m-%d')) & (
                                            meteorological_data['Date'] <= pd.to_datetime(f'{i}-02-10',
                                                                                          format='%Y-%m-%d'))]['degree_hours_10_min'].sum()
                              , "T_Flowering":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-02-11',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime( f'{i}-03-31',format='%Y-%m-%d'))]
                               ['degree_hours_10_min'].sum()

                              , "T_Thinning":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-04-01',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime(f'{i}-05-15',format='%Y-%m-%d'))]
                               ['degree_hours_10_min'].sum()

                              , "T_Growth":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-05-16',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime(f'{i}-07-31',
                                                           format='%Y-%m-%d'))]
                               ['degree_hours_10_min'].sum()

                              , "T_June_Drop":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-06-01',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime(f'{i}-06-30',
                                                           format='%Y-%m-%d'))]
                               ['degree_hours_10_min'].sum()

                              , "T_Ripening":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-08-01',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime(f'{i}-08-31',format='%Y-%m-%d'))]
                               ['degree_hours_10_min'].sum()

                              , "T_Harvest":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-09-01',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime(f'{i}-10-31',format='%Y-%m-%d'))]
                               ['degree_hours_10_min'].sum()

                               , "H_Inf_differentiation":
                                meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i - 1}-11-01',format='%Y-%m-%d')) & (
                                                        meteorological_data['Date'] <= pd.to_datetime(f'{i}-02-10',format='%Y-%m-%d'))]
                                ['Relative humidity (%)'].mean()

                                , "H_Flowering":
                                meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-02-11', format='%Y-%m-%d')) & (
                                                                meteorological_data['Date'] <= pd.to_datetime(f'{i}-03-31', format='%Y-%m-%d'))]
                                ['Relative humidity (%)'].mean()

                                , "H_Thinning":
                                meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-04-01', format='%Y-%m-%d')) & (
                                                                meteorological_data['Date'] <= pd.to_datetime(f'{i}-05-15',format='%Y-%m-%d'))]
                                ['Relative humidity (%)'].mean()

                                , "H_Growth":
                                meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-05-16', format='%Y-%m-%d')) & (
                                                                            meteorological_data['Date'] <= pd.to_datetime(f'{i}-07-31', format='%Y-%m-%d'))]
                                ['Relative humidity (%)'].mean()

                                , "H_June_Drop":
                                meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-06-01', format='%Y-%m-%d')) & (
                                                                            meteorological_data['Date'] <= pd.to_datetime(f'{i}-06-30',format='%Y-%m-%d'))]
                                ['Relative humidity (%)'].mean()

                                , "H_Ripening":
                                meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-08-01', format='%Y-%m-%d')) & (
                                                                            meteorological_data['Date'] <= pd.to_datetime(f'{i}-08-31',format='%Y-%m-%d'))]
                                ['Relative humidity (%)'].mean()

                                , "H_Harvest":
                                meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-09-01', format='%Y-%m-%d')) & (
                                                                            meteorological_data['Date'] <= pd.to_datetime(f'{i}-10-31',format='%Y-%m-%d'))]
                                ['Relative humidity (%)'].mean()
                                
                              , "E_Inf_differentiation": meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i-1}-11-01',
                                                                               format='%Y-%m-%d')) & (
                                            meteorological_data['Date'] <= pd.to_datetime(f'{i}-02-10',
                                                                                          format='%Y-%m-%d'))]['Evaporation (mm/10min)'].sum()
                              , "E_Flowering":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-02-11',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime( f'{i}-03-31',format='%Y-%m-%d'))]
                               ['Evaporation (mm/10min)'].sum()

                              , "E_Thinning":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-04-01',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime(f'{i}-05-15',format='%Y-%m-%d'))]
                               ['Evaporation (mm/10min)'].sum()

                              , "E_Growth":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-05-16',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime(f'{i}-07-31',
                                                           format='%Y-%m-%d'))]
                               ['Evaporation (mm/10min)'].sum()

                              , "E_June_Drop":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-06-01',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime(f'{i}-06-30',
                                                           format='%Y-%m-%d'))]
                               ['Evaporation (mm/10min)'].sum()

                              , "E_Ripening":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-08-01',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime(f'{i}-08-31',format='%Y-%m-%d'))]
                               ['Evaporation (mm/10min)'].sum()

                              , "E_Harvest":
                               meteorological_data.loc[(meteorological_data['Date'] >= pd.to_datetime(f'{i}-09-01',format='%Y-%m-%d')) & (
                                                               meteorological_data['Date'] <= pd.to_datetime(f'{i}-10-31',format='%Y-%m-%d'))]
                               ['Evaporation (mm/10min)'].sum()  }])], ignore_index=True)


        print('stop')

    #  df = pd.merge(processed_df_weather, df_periods, on="Year")
    return df_periods #df

In [67]:
df_meteorology = weather_data()

a
b


In [73]:
df_meteorology.to_excel(path_files + "\\Processed data\\Combined Meteorological data 2011 2023 radiation.xlsx")

In [74]:
meteorological_data = pd.read_excel(path_files + "\\Processed data\\Combined Meteorological data 2011 2023.xlsx")


In [75]:
# Constants
rho = 1.2  # Air density (kg/m³)
cp = 1013  # Specific heat of air (J/kg°C)
lambda_v = 2.45 * 10**6  # Latent heat of vaporization (J/kg)
gamma = 0.065  # Psychrometric constant (kPa/°C)

# Function to calculate saturation vapor pressure (es) in kPa
def saturation_vapor_pressure(T):
    return 0.6108 * np.exp((17.27 * T) / (T + 237.3))

# Function to calculate actual vapor pressure (ea) in kPa
def actual_vapor_pressure(RH, T):
    es = saturation_vapor_pressure(T)
    return (RH / 100) * es

# Function to calculate the slope of the saturation vapor pressure curve (Delta) in kPa/°C
def delta_slope(T):
    es = saturation_vapor_pressure(T)
    return (4098 * es) / ((T + 237.3) ** 2)

# Function to calculate Penman-Monteith Evaporation (mm/day)
def penman_monteith_evaporation(Rn, T, RH):
    # Assumed: G = 0 (soil heat flux density is negligible)
    G = 0
    
    # Convert Radiation from W/m² to MJ/m²/10min
    Rn_MJ_m2_10min = (Rn / 1000000) * 600
    
    # Calculate the saturation and actual vapor pressures
    es = saturation_vapor_pressure(T)
    ea = actual_vapor_pressure(RH, T)
    
    # Calculate the slope of the saturation vapor pressure curve
    delta = delta_slope(T)
    
    # Calculate the Penman-Monteith evaporation (mm/10min)
    numerator = delta * (Rn_MJ_m2_10min - G) + rho * cp * (es - ea) / lambda_v
    denominator = delta + gamma * (1 + 0)  # Assuming r_s/r_a = 0 for simplicity
    
    E_t = numerator / denominator
    return E_t

# Replace '-' with NaN
meteorological_data['Radiation (W/m2)'] = pd.to_numeric(meteorological_data['Radiation (W/m2)'], errors='coerce')
meteorological_data['Temperature (°C)'] = pd.to_numeric(meteorological_data['Temperature (°C)'], errors='coerce')
meteorological_data['Relative humidity (%)'] = pd.to_numeric(meteorological_data['Relative humidity (%)'], errors='coerce')


# Replace negative values in specific columns
columns_to_check = ['Radiation (W/m2)', 'Relative humidity (%)']
meteorological_data[columns_to_check] = meteorological_data[columns_to_check].applymap(lambda x: np.nan if x < 0 else x)


# Assuming your dataframe has the required columns: 'Radiation (W/m2)', 'Temperature (°C)', 'Relative humidity (%)'
meteorological_data['Evaporation (mm/10min)'] = meteorological_data.apply(lambda row: penman_monteith_evaporation(
    row['Radiation (W/m2)'], row['Temperature (°C)'], row['Relative humidity (%)']
), axis=1)

# Now df contains a new column 'Evaporation (mm/day)' with the calculated evaporation
    


C:\Users\salzer\AppData\Local\Temp\ipykernel_15016\854858161.py:51: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  meteorological_data[columns_to_check] = meteorological_data[columns_to_check].applymap(lambda x: np.nan if x < 0 else x)


In [77]:
meteorological_data.head()

,Unnamed: 0,Station,Date & Time (Winter),Radiation (W/m2),Relative humidity (%),Temperature (°C),Date,Time,hour,month,year,Evaporation (mm/10min)
0,0,יוטבתה,01/01/2011 00:00,NaN,64.0,14.2,2011-01-01,00:00:00,0,1,2011,NaN
1,1,יוטבתה,01/01/2011 00:10,NaN,68.0,13.3,2011-01-01,00:10:00,0,1,2011,NaN
2,2,יוטבתה,01/01/2011 00:20,NaN,69.0,13.1,2011-01-01,00:20:00,0,1,2011,NaN
3,3,יוטבתה,01/01/2011 00:30,NaN,71.0,12.5,2011-01-01,00:30:00,0,1,2011,NaN
4,4,יוטבתה,01/01/2011 00:40,NaN,73.0,12.2,2011-01-01,00:40:00,0,1,2011,NaN


In [80]:
df_meteorology.to_excel(path_files + "\\Processed data\\Combined Meteorological data 2011 2023 radiation.xlsx")

In [78]:
df_meteorology_periods = weather_data_Flowering_periods(meteorological_data)

stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop


In [81]:
df_meteorology_periods.to_excel(path_files + "\\Processed data\\Meteorological data 2011 2023 periods.xlsx")

### Function: yield_processing

In [17]:
def yield_processing(processed_df):

    yield_df = pd.read_excel(path_files + "\\Processed data\\Yield 2015-2023.xlsx", header=14)


    for index, row in processed_df.iterrows():
        yield_temp = yield_df.loc[yield_df['HAVA:PLOT'] == processed_df.at[index, "Farm:Plot"]]
        if not yield_temp.empty:
            if processed_df.at[index, 'Year'] != 2012:
                yield_temp = yield_temp.fillna(0)
                processed_df.at[index, 'Yield'] = yield_temp[f"{processed_df.at[index, 'Year']} FINAL"].values[0]
        else:
            processed_df.at[index, 'Yield'] = float("NaN")
        processed_df.loc[processed_df['Yield'] == 0, 'Yield'] = float("NaN")

    #Relative number of trees, then yield per tree
    num_trees = pd.read_excel(path_files + "\\Processed data\\Trees per plot.xlsx")
    num_trees['Age'] =  num_trees['גיל העץ']
    num_trees.loc[(num_trees['Age'] < 4)  , 'percent_per_age_group'] = 0 #according to table recived by Tamarika
    num_trees.loc[(num_trees['Age'] >= 4)& (num_trees['Age'] <= 6), 'percent_per_age_group'] = 20/150
    num_trees.loc[(num_trees['Age'] >= 7)& (num_trees['Age'] <= 9), 'percent_per_age_group'] = 50/150
    num_trees.loc[(num_trees['Age'] >= 10)& (num_trees['Age'] <= 12), 'percent_per_age_group'] = 80/150
    num_trees.loc[(num_trees['Age'] >= 13)& (num_trees['Age'] <= 15), 'percent_per_age_group'] = 110/150
    num_trees.loc[(num_trees['Age'] >= 16)& (num_trees['Age'] <= 19), 'percent_per_age_group'] = 136/150
    num_trees.loc[(num_trees['Age'] >= 20), 'percent_per_age_group'] = 150/150
    num_trees['Relative_number_of_adult_trees'] = num_trees['כמות עצים'] * num_trees['percent_per_age_group']

    num_trees.rename(columns={'כמות עצים': 'Number_of_trees'}, inplace=True)
    num_trees.rename(columns={'HAVA:PLOT':'Farm:Plot'}, inplace=True)

    num_trees_grouped = num_trees.groupby(['Farm:Plot']).Number_of_trees.sum().reset_index()
    num_trees_grouped.rename(columns={'Number_of_trees':'Number_of_trees_plot'}, inplace=True)
    processed_df = pd.merge(processed_df, num_trees_grouped, on='Farm:Plot', how='inner')


    num_trees_grouped = num_trees.groupby(['Farm:Plot']).Relative_number_of_adult_trees.sum().reset_index()
    processed_df = pd.merge(processed_df, num_trees_grouped, on ='Farm:Plot', how='inner')

    processed_df.loc[processed_df['Relative_number_of_adult_trees'] != 0, 'Relative_yield_per_adult_tree'] = processed_df['Yield'] / processed_df['Relative_number_of_adult_trees']
    processed_df = processed_df.loc[:, ~processed_df.columns.str.contains('^Unnamed')]

    # Relative contibution to yield for different sections of each plot (different ages of trees)

    num_trees['Weight_age_group'] = num_trees['percent_per_age_group'] * 150

    num_trees_processed = pd.DataFrame()

    for plot in num_trees['Farm:Plot'].unique():
        num_trees_temp = num_trees.loc[num_trees['Farm:Plot'] == plot]
        num_trees_temp["total_weight"] = num_trees_temp['Weight_age_group'] * num_trees_temp['Number_of_trees']
        num_trees_temp['Relative_section_contribution_to_yield'] =  num_trees_temp["total_weight"] /  num_trees_temp["total_weight"].sum()
        num_trees_temp = num_trees_temp[['Farm:Plot', 'Age', 'Weight_age_group', 'Number_of_trees', 'Relative_section_contribution_to_yield']]
        num_trees_temp.rename(columns={'Number_of_trees':'Number_of_trees_section'}, inplace=True)
        num_trees_processed = num_trees_processed.append(num_trees_temp, ignore_index=True)


    processed_df = pd.merge(processed_df, num_trees_processed, on='Farm:Plot', how='inner')
    processed_df['Relative_yield_from_section'] = processed_df['Yield'] * (processed_df['Relative_section_contribution_to_yield'])
    processed_df['Relative_yield_per_tree'] = processed_df['Relative_yield_from_section'] / (processed_df['Number_of_trees_section'])
    processed_df[['Number_of_trees_plot','Relative_number_of_adult_trees','Relative_yield_per_adult_tree', 'Number_of_trees_section', 'Relative_yield_per_tree']].replace(0, np.nan, inplace=True)
    return processed_df

### Function: Quality processing    

In [ ]:

def quality_processing(processed_df):

    processed_df['Farm'] = [x.partition(":")[0] for x in processed_df['Farm:Plot']]
    quality_df = pd.read_csv(path_files + "\\Raw data\\df_shilpuach_weight_quality_2012_2023.csv")

    quality_df['Farm'] = quality_df['Farm'].astype(str)
    processed_df = pd.merge(processed_df, quality_df, on=["Farm", "Year"], how="left")

    for column in quality_df.columns:
        if column not in ['Farm', 'Year']:
            new_column = f"{column} from yield"
            processed_df[new_column] = processed_df[column] * processed_df['Relative_yield_per_tree']
    return processed_df


### Function: create homogenic df

In [19]:

def create_homogemic_df(processed_data):
    one_section_df = pd.DataFrame()
    for plot in processed_data['Farm:Plot'].unique():
        num_trees_temp = processed_data.loc[processed_data['Farm:Plot'] == plot]
        for Date in num_trees_temp['Date'].unique():
            temp = num_trees_temp.loc[num_trees_temp['Date'] == Date]
            if temp.shape[0] == 1:
                one_section_df = one_section_df.append(temp, ignore_index=True)
    one_section_df.to_excel(path_files + "\\Processed data\\Homogenic plots 2012-2023.xlsx")


## Modeling

### Model: features and labels

In [4]:
monthly_temperature = ['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12']
monthly_humidity = ['h1','h2','h3','h4','h5','h6','h7','h8','h9','h10', 'h11','h12']
bloom_periods_temperature = ['T_Sorting', 'T_Flowering', 'T_Diluting', 'T_Growing', 'T_June_Dropping', 'T_Ripening', 'T_Harvest']
bloom_periods_humidity = ['H_Sorting', 'H_Flowering', 'H_Diluting', 'H_Growing', 'H_June_Dropping', 'H_Ripening', 'H_Harvest']
tree_age = ['Age']
counts = ['AVG_clusters', 'AVG_fruitlets_upper', 'AVG_fruitlets_center', 'AVG_fruitlets_lower', 'SUM_tree_fruitlets', 'Days since january 1st']

monthly_weather = []
monthly_weather.extend(monthly_temperature)
monthly_weather.extend(monthly_humidity)


bloom_periods_weather = []
bloom_periods_weather.extend(bloom_periods_temperature)
bloom_periods_weather.extend(bloom_periods_humidity)

all_features = []
all_features.extend(bloom_periods_temperature)
all_features.extend(bloom_periods_humidity)
all_features.extend(counts)
all_features_without_monthly = all_features.copy()
all_features.extend(monthly_temperature)
all_features.extend(monthly_humidity)

quality_labels = ['0% to 5% from yield','25% to 40% from yield', '5% to 25% from yield','Other_shilpuah from yield', '15gr to 18gr from yield',
                  '18gr to 23gr from yield', '23gr or more from yield', 'Other_weight from yield', 'small from yield']


feature_dict = {'All features without monthly weather': all_features_without_monthly,
                'Development periods weather': bloom_periods_weather,  'Monthly weather': monthly_weather,
                'Counts data':counts, 'All features': all_features}


feature_dict_with_age = {'All features without monthly weather': all_features_without_monthly, 'tree age':tree_age,
                'Development periods weather': bloom_periods_weather,  'Monthly weather': monthly_weather,
                'Counts data':counts, 'Counts and tree age':counts, 'All features': all_features}


labels = ['Relative_yield_per_adult_tree', 'Relative_yield_per_tree']

### Model execution:

In [ ]:

def basic_model(data, feature_dict, labels):
    df_results = pd.DataFrame()
    for feature_set in feature_dict:
        column_list = feature_dict[feature_set].copy()
        column_list.extend(labels)
        df = data[column_list].dropna()
        print("\nSet:", feature_set)
        X = df[feature_dict[feature_set]]
        print("Number of samples:", X.shape[0])
        Y = df[labels]
        for label in Y:
            print(f"\n{label} using {feature_set}:\n")

            for model in [xgb.XGBRegressor(verbosity=0), RandomForestRegressor(random_state=42), LinearRegression()]:
                model_str = str(model)
                if 'XGBRegressor' in model_str:
                    model_str = 'XGBRegressor'

                y = Y[label]

                cv = KFold(n_splits=5, random_state=42, shuffle=True)

                scores_r2 = cross_val_score(model, X, y, scoring='r2',
                                            cv=cv, n_jobs=-1)

                scores_rmse = cross_val_score(model, X, y,
                                              scoring='neg_root_mean_squared_error',
                                              cv=cv, n_jobs=-1)
                scores_rmse = scores_rmse * -1

                print(f"{model_str} without reduction:\nR2: {np.average(scores_r2)}, RMSE: { np.average(scores_rmse)}")

                df_results = df_results.append(
                    {'Feature set': feature_set, 'Label': [label], 'Model': model_str,
                     'r2': np.average(scores_r2),
                     'RMSE': np.average(scores_rmse),
                     'Num features': len(X.columns),
                     'features': X.columns.values.tolist(),
                     'feature_reduction': 'no', 'Data points': X.shape[0]}, ignore_index=True)

                if feature_set != 'Baseline':
                    fft = SequentialFeatureSelector(estimator=model, k_features='best', forward=True, scoring='r2',
                                                    cv=cv)

                    fft.fit(X, y)
                    names_after_reduction = fft.k_feature_names_  # Chosen features after reduction
                    names_after_reduction = list(names_after_reduction)

                    scores_r2 = cross_val_score(model, X[names_after_reduction], y, scoring='r2',
                                                cv=cv,
                                                n_jobs=-1)  # Running the model on the chosen features from feature reduction
                    scores_rmse = cross_val_score(model, X[names_after_reduction], y,
                                                  scoring='neg_root_mean_squared_error',
                                                  cv=cv, n_jobs=-1)
                    scores_rmse = scores_rmse * -1

                    print(f"{model_str} with reduction:\nR2: {np.average(scores_r2)}, RMSE: { np.average(scores_rmse)}")

                    df_results = df_results.append(
                        {'Feature set': feature_set, 'Label': [label], 'Model': model_str,
                         'r2': np.average(scores_r2),
                         'RMSE': np.average(scores_rmse),
                         'Num features': len(fft.k_feature_idx_),
                         'features': names_after_reduction,
                         'feature_reduction': 'yes', 'Data points': X.shape[0]}, ignore_index=True)
                    # prediction = cross_val_predict(model, X[names_after_reduction], y, cv=cv)
                    # prediction_df = pd.DataFrame()
                    # prediction_df2 = pd.DataFrame(
                    #     columns=['Label', 'Real_value', 'Prediction'])
                    # prediction_df2['Real_value'] = y
                    # prediction_df2['Prediction'] = prediction
                    #
                    # prediction_df2['Label'] = label
                    #
                    # prediction_df = pd.concat([prediction_df, prediction_df2])
                    #
                    # fig = plt.figure(figsize=(15, 10))
                    # ax = sns.scatterplot(x=prediction_df['Real_value'], y=prediction_df['Prediction'], s=200)
                    #
                    # plt.xlabel(f'True value', fontsize=20)
                    # plt.ylabel(f'Predicted value', fontsize=20)
                    # lineStart = prediction_df.Real_value.min() - 0.1
                    # lineEnd = prediction_df.Real_value.max() + 0.1
                    # plt.plot([lineStart, lineEnd], [lineStart, lineEnd], 'k-', linestyle='dashed')
                    # plt.suptitle(f"True vs. Predicted tree yield", fontsize=25)
                    # plt.xlim(lineStart, lineEnd)
                    # plt.ylim(lineStart, lineEnd)
                    # plt.show()

    return df_results

def Feature_analysis(data, features, label):

        column_list = features.copy()
        column_list.append(label)
        df = data[column_list].dropna()

        X = df[features]
        # corr tables
        corrMatrix = df.corr()
        mask = np.triu(np.ones_like(corrMatrix, dtype=bool))

        ax = sns.heatmap(corrMatrix, annot_kws={"size": 10}, mask=mask, annot=True, vmin=-1, vmax=1)
        # plt.suptitle(f'Correlations between {feature_set} and yield per tree', fontsize=35)
        ax.set_xticklabels(list(corrMatrix.columns), rotation=30, fontsize=10)
        ax.set_yticklabels(list(corrMatrix.columns), rotation=20, fontsize=10)
        # plt.gca().invert_xaxis()
        plt.show()

        # SHAP
        y = df[label]
        shap.initjs()
        xgb_shap = xgb.XGBRegressor(verbosity=0).fit(X, y)
        explainer = shap.TreeExplainer(xgb_shap)
        shap_values = explainer.shap_values(X)

        shap.summary_plot(shap_values, X, show=False, plot_type="violin")
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)

        # plt.suptitle(f"Feature importance for {label} prediction - {feature_set} set - 2024")
        plt.show()


def age_model(data, feature_dict, label):
    df_results = pd.DataFrame()
    for feature_set in feature_dict:
        feature_list = feature_dict[feature_set].copy()
        if feature_set in ['All features without monthly weather', 'All features', 'Counts and tree age']:
            feature_list.append('Age')
        column_list = feature_list.copy()
        column_list.extend(labels)
        df = data[column_list].dropna()
        print("\nSet:", feature_set)
        X = df[feature_list]
        print("Number of samples:", X.shape[0])
        Y = df[labels]
        for label in Y:
            print(f"\n{label} using {feature_set}:\n")
            for model in [xgb.XGBRegressor(verbosity=0), RandomForestRegressor(random_state=42), LinearRegression()]:
                model_str = str(model)
                y = Y[label]
                if 'XGBRegressor' in model_str:
                    model_str = 'XGBRegressor'
                cv = KFold(n_splits=5, random_state=42, shuffle=True)

                scores_r2 = cross_val_score(model, X, y, scoring='r2',
                                            cv=cv, n_jobs=-1)

                scores_rmse = cross_val_score(model, X, y,
                                              scoring='neg_root_mean_squared_error',
                                              cv=cv, n_jobs=-1)
                scores_rmse = scores_rmse * -1

                print(f"{model_str} without reduction:\nR2: {np.average(scores_r2)}, RMSE: {np.average(scores_rmse)}")

                df_results = df_results.append(
                    {'Feature set': feature_set, 'Label': [label], 'Model': model_str,
                     'r2': np.average(scores_r2),
                     'RMSE': np.average(scores_rmse),
                     'Num features': len(X.columns),
                     'features': X.columns.values.tolist(),
                     'feature_reduction': 'no', 'Data points': X.shape[0]}, ignore_index=True)
                if feature_set != 'Baseline':
                    fft = SequentialFeatureSelector(estimator=model, k_features='best', forward=True, scoring='r2',
                                                    cv=cv)

                    fft.fit(X, y)
                    names_after_reduction = fft.k_feature_names_  # Chosen features after reduction
                    names_after_reduction = list(names_after_reduction)

                    scores_r2 = cross_val_score(model, X[names_after_reduction], y, scoring='r2',
                                                cv=cv,
                                                n_jobs=-1)  # Running the model on the chosen features from feature reduction
                    scores_rmse = cross_val_score(model, X[names_after_reduction], y,
                                                  scoring='neg_root_mean_squared_error',
                                                  cv=cv, n_jobs=-1)
                    scores_rmse = scores_rmse * -1

                    print(f"{model_str} with reduction:\nR2: {np.average(scores_r2)}, RMSE: {np.average(scores_rmse)}")

                    df_results = df_results.append(
                        {'Feature set': feature_set, 'Label': [label], 'Model': model_str,
                         'r2': np.average(scores_r2),
                         'RMSE': np.average(scores_rmse),
                         'Num features': len(fft.k_feature_idx_),
                         'features': names_after_reduction,
                         'feature_reduction': 'yes', 'Data points': X.shape[0]}, ignore_index=True)
                    #
                    # prediction = cross_val_predict(model, X[names_after_reduction], y, cv=cv)
                    # prediction_df = pd.DataFrame()
                    # prediction_df2 = pd.DataFrame(
                    #     columns=['Label', 'Real_value', 'Prediction'])
                    # prediction_df2['Real_value'] = y
                    # prediction_df2['Prediction'] = prediction
                    #
                    # prediction_df2['Label'] = label
                    #
                    # prediction_df = pd.concat([prediction_df, prediction_df2])
                    #
                    #
                    # fig = plt.figure(figsize=(15, 10))
                    # ax = sns.scatterplot(x=prediction_df['Real_value'], y=prediction_df['Prediction'], s=200)
                    #
                    # plt.xlabel(f'True value', fontsize=20)
                    # plt.ylabel(f'Predicted value', fontsize=20)
                    # lineStart = prediction_df.Real_value.min() - 0.1
                    # lineEnd = prediction_df.Real_value.max() + 0.1
                    # plt.plot([lineStart, lineEnd], [lineStart, lineEnd], 'k-', linestyle='dashed')
                    # plt.suptitle(f"True vs. Predicted tree yield", fontsize=25)
                    # plt.xlim(lineStart, lineEnd)
                    #
                    # plt.ylim(lineStart, lineEnd)
                    # plt.show()
    return df_results


# Functions

### Function: lower_and_upper_error_lists

In [ ]:
def lower_and_upper_error_lists (df):
  '''
  this function returns a list of the upper and lower arror of each prediction. used for the error bar in the prediction scatter plot
  '''
  lower = []
  upper = []
  for i in range(len(df['Prediction'])):
    true_label = df['Real_value'][i]
    pred = df['Prediction'][i]
    error = true_label - pred
    if error > 0: #true label > pred : pred had a lower error
      lower.append(error)
      upper.append(0)
    elif error < 0: #pred had an upper error: true<pred
      lower.append(0)
      upper.append(-error)
    else: #pred was right on the money
      lower.append(0)
      upper.append(0)
  lower = [x // 4 for x in lower]
  upper =  [x // 4 for x in upper]
  return lower, upper